<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Table-Creation" data-toc-modified-id="Table-Creation-1">Table Creation</a></span></li><li><span><a href="#DataBase-creation" data-toc-modified-id="DataBase-creation-2">DataBase creation</a></span></li><li><span><a href="#Querying-the-database" data-toc-modified-id="Querying-the-database-3">Querying the database</a></span></li><li><span><a href="#Deployment" data-toc-modified-id="Deployment-4">Deployment</a></span></li></ul></div>

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

### Table Creation

In [ ]:
# Retrieving data for table1

In [4]:
base_url = 'https://books.toscrape.com/catalogue/'

In [31]:
titles = []
stars = []
prices = []
pic_urls = []

In [32]:
for page in range(1,3):
    
    page_url = urljoin(base_url, f'page-{page}.html')
    
    response = requests.get(page_url)
    response.status_code
    
    if response.status_code == 200:
        print(f'Scraping page #{page} at url: {page_url}.')
        html = response.content
        soup = BeautifulSoup(html, 'lxml')
        uls = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')
        pages_titles = []
        pages_stars = []
        pages_prices = []
        page_pic_urls = []
        
        for ul in uls:
            title = ul.find_all('a')[1]['title']
            pages_titles.append(title)  
        
            star = ul.find_all('p')[0].get('class')[1]
            pages_stars.append(star)
            
            price = ul.find_all('p')[1].text
            pages_prices.append(price)
            
            pic_url_ending = ul.find_all('img')[0]['src']
            full_pic_url = base_url+pic_url_ending
            page_pic_urls.append(full_pic_url)
            
        
        titles.extend(pages_titles)
        stars.extend(pages_stars)
        prices.extend(pages_prices)
        pic_urls.extend(page_pic_urls)
        
    else:
        print(f'Url #{i} ({page_url}) is throwing an error. Skipping to url #{i+1}')

Scraping page #1 at url: https://books.toscrape.com/catalogue/page-1.html.
Scraping page #2 at url: https://books.toscrape.com/catalogue/page-2.html.


In [ ]:
# Creating table1

In [34]:
table1 = pd.DataFrame({'book_id': list(range(100, 100+len(titles))),
                       'title': titles,
                       'star_rating': stars,
                       'price_GBP': prices,
                       'pic_url': pic_urls})

In [37]:
table1.price_GBP = table1.price_GBP.map(lambda x: float(x.split('£')[1]))

In [48]:
table1['price_EUR'] = round(table1.price_GBP.map(lambda x: x*1.19),2)

In [49]:
table1

,book_id,title,star_rating,price_GBP,pic_url,price_EUR
0,100,A Light in the Attic,Three,51.77,https://books.toscrape.com/catalogue/../media/...,61.61
1,101,Tipping the Velvet,One,53.74,https://books.toscrape.com/catalogue/../media/...,63.95
2,102,Soumission,One,50.10,https://books.toscrape.com/catalogue/../media/...,59.62
3,103,Sharp Objects,Four,47.82,https://books.toscrape.com/catalogue/../media/...,56.91
4,104,Sapiens: A Brief History of Humankind,Five,54.23,https://books.toscrape.com/catalogue/../media/...,64.53
5,105,The Requiem Red,One,22.65,https://books.toscrape.com/catalogue/../media/...,26.95
6,106,The Dirty Little Secrets of Getting Your Dream...,Four,33.34,https://books.toscrape.com/catalogue/../media/...,39.67
7,107,The Coming Woman: A Novel Based on the Life of...,Three,17.93,https://books.toscrape.com/catalogue/../media/...,21.34
8,108,The Boys in the Boat: Nine Americans and Their...,Four,22.60,https://books.toscrape.com/catalogue/../media/...,26.89
9,109,The Black Maria,One,52.15,https://books.toscrape.com/catalogue/../media/...,62.06


In [ ]:
table1.to_csv('table1.csv')

In [ ]:
# Retrieving data for table2

In [42]:
texts = []
for title in titles:
    print(f'retrieving text for "{title}"')
    r = requests.post('https://api.deepai.org/api/text-generator',
                      data={'text': f'The text of {title} is',},
                      headers={'api-key': 'quickstart-QUdJIGlzIGNvbWluZy4uLi4K'})
    text = r.json()['output']
    texts.append(text)
    

retrieving text for "A Light in the Attic"


KeyError: 'output'

In [ ]:
# Creating table2

In [ ]:
table2 = pd.DataFrame({'book_id': list(range(100, 100+len(titles))),
                       'title': titles,
                       'text': [texts])

In [43]:
r.json()

{'status': "Looks like you're enjoying our API. Want to keep using it? Sign up to get an API Key that's as unique as you are. It's free! https://deepai.org/"}

In [ ]:
table2.to_csv('table2.csv')

In [ ]:
# Retrieving data for table3

In [ ]:
# So, the final code for file download is
url = "https://file-examples.com/wp-content/uploads/2017/04/file_example_MP4_1920_18MG.mp4"

with requests.get(url, stream = True) as r:
    with open("Sample_video_18_MB.mp4", "wb") as f:
        for chunk in r.iter_content(chunk_size = 16*1024):
            f.write(chunk)

### DataBase creation

### Querying the database

### Deployment